Case
===

http://blog.mywarwithentropy.com/search/label/SpyHunter

(c) Bret Jordan, 2014, Educational use permitted, see license for details

@adricnet worked thorugh this old challenge for practice. You may want to give it a shot before reading on as there are **spoilers** for the plot throughout.

Scenario:
---

Some questions and clues without any any specific indicators, PDF and PCAP available:

http://blog.mywarwithentropy.com/2014/11/spy-hunter-holiday-challenge-2014.html

Evidence:
---

A zipped pcap and MD5 hash 

Methodology:
---

1. Classifiers and meta
2. Slice and dice
3. Focus in on some *spoiler* chat traffic

Analysis
===

Initial evidence processing and classification
---

* capinfos
* tshark
* tcpdump
* tcpflow
* bro

```bash
## Get pcap file meta
capinfos > hermes-capinfos.txt

## Get some conversation tables from tshark, straight from `man 1 tshark`
tshark -nn -r SpyHunter-Operation_Hermes-ver1.pcap -z conv,eth -q > spyh/hermes-ethconv
tshark -nn -r SpyHunter-Operation_Hermes-ver1.pcap -z conv,tcp -q > spyh/hermes-tcpconv
tshark -nn -r SpyHunter-Operation_Hermes-ver1.pcap -z conv,udp -q > spyh/hermes-udpconv
```

The conversation lists showed a great deal more DNS conversations than TCP or Ethernet ones, so after nosing around briefly in the tcpdump decodes for port 53 traffic I went back to tshark to do a quick frequency sort on the DNS names

```bash
tcpdump -nnr SpyHunter-Operation_Hermes-ver1.pcap -c2 'port 53'
tcpdump -nnr SpyHunter-Operation_Hermes-ver1.pcap -c5 'port 53' 
tcpdump -nnr SpyHunter-Operation_Hermes-ver1.pcap -c24 'port 53' 
```

I ran a quick experiment with some likely DNS packets to get the field names for tshark DNS fields

```bash
tcpdump -nnr SpyHunter-Operation_Hermes-ver1.pcap -c24 -w hermes-53-packets.pcap 'port 53' 
wireshark hermes-53-packets.pcap 
## click around in wireshark to read the field names from the status bar, try a couple display filters...
tshark -r hermes-53-packets.pcap -Tfields -e dns.resp.name -e dns.resp.type | more
tshark -r hermes-53-packets.pcap -Tfields -e dns.resp.name -e dns.resp.type -Y dns | more
tshark -r hermes-53-packets.pcap -Tfields -e dns.qry.name -e dns.resp.name -e dns.resp.type -Y dns | more
tshark -r hermes-53-packets.pcap -Tfields -e dns.qry.name -e dns.resp.name -e dns.resp.type -Y dns 
```

Then I pulled the DNS query names from all of the DNS traffic in the complete pcap and counted.

```bash
tshark -r SpyHunter-Operation_Hermes-ver1.pcap -Tfields -e dns.qry.name -Y dns 
tshark -r SpyHunter-Operation_Hermes-ver1.pcap -Tfields -e dns.qry.name -Y dns > hermes-dns-query-names

##
sort hermes-dns-query-names | uniq -c | sort -rn > hermes-dns-query-counts  ## most frequent
head hermes-dns-query-counts ## 10 most frequent
sort hermes-dns-query-names | uniq -c | sort -n | head ## 10 least frequent
```

That didn't generate any strong leads but did confirm that there is a lot of web "surfing" traffic in the pcap.
I also ran the pcap through tcpflow :

```bash
## Use the http decoder to get html files and images as well as text decodes from the conversations
## Use Fm to split into folders by Megabytes to avoid putting 10K+ files in a folder
tcpflow -o hermes/ -r SpyHunter-Operation_Hermes-ver1.pcap -e http -Fm
## then a regualr run to get the non-http streams
tcpflow -o hermes/ -r SpyHunter-Operation_Hermes-ver1.pcap -Fm
```

TCPflow decoded out many many conversations and files. Taking a quick peek for PE files didn't net any, but did find two downloaded ZIP archives

```bash
cd hermes/000000-000999/
for folder in 0000* ; do file $folder/* | grep -i intel; done
for folder in 0000* ; do file $folder/* | grep -i zip; done

000008/069.160.243.051.00080-192.168.000.062.53249􀀜-HTTPBODY-001:      Zip archive data, at least v1.0 to extract
000009/074.050.101.106.00080-192.168.000.062.53742􀀜-HTTPBODY-001:      Zip archive data, at least v2.0 to extract
000011/092.049.128.010.00080-192.168.000.062.51625􀀜-HTTPBODY-001.zip:  Zip archive data, at least v1.0 to extract
000016/014.144.001.039.00080-192.168.000.062.51642􀀜-HTTPBODY-001.zip:  Zip archive data, at least v1.0 to extract
000016/014.144.001.039.00080-192.168.000.062.51643􀀜-HTTPBODY-001.zip:  Zip archive data, at least v1.0 to extract
```

First two are steghide downloads ?! That may be a problem... Third is pretty suspicious:

```bash
unzip -l 000008/069.160.243.051.00080-192.168.000.062.53249<U+10001C>-HTTPBODY-001
unzip -l 000009/074.050.101.106.00080-192.168.000.062.53742<U+10001C>-HTTPBODY-001
...
    6506  10-15-03 03:33   steghide/README.txt
   595968  10-15-03 05:44   steghide/steghide.exe
      814  10-15-03 03:33   steghide/todo.txt
 --------                   -------
  3931066                   33 files

unzip -l 000011/092.049.128.010.00080-192.168.000.062.51625􀀜-HTTPBODY-001.zip
Archive:  000011/092.049.128.010.00080-192.168.000.062.51625􀀜-HTTPBODY-001.zip
  Length     Date   Time    Name
 --------    ----   ----    ----
        0  10-09-14 22:12   Sample/
   111157  10-09-14 21:59   Sample/image_manager__content_full _tscherenkov.jpg
    44356  10-09-14 21:59   Sample/Mitsubishi PWR nuclear fuel assembly schematic.jpg
       88  10-09-14 22:12   Sample/README
 --------                   -------
   155601                   4 files

unzip -l 000016/014.144.001.039.00080-192.168.000.062.51642􀀜-HTTPBODY-001.zip
Archive:  000016/014.144.001.039.00080-192.168.000.062.51642􀀜-HTTPBODY-001.zip
  Length     Date   Time    Name
 --------    ----   ----    ----
        0  10-09-14 23:56   README/
      113  10-09-14 23:56   README/README
 --------                   -------
      113                   2 files
 
unzip -l 000016/014.144.001.039.00080-192.168.000.062.51643􀀜-HTTPBODY-001.zip  
... 
   108701  09-22-14 23:19   Data/ssb1-module6.jpg
   120877  09-22-14 23:30   Data/tunnel3-4.jpg
   114088  09-22-14 23:38   Data/upgrade.jpg
   207947  10-09-14 21:56   Data/vs4effcfa2.jpg
 --------                   -------
 12952570                   37 files

vi hermes-steghide-dls ## make notes

```

and I ran the whole PCAP through Bro IDS as well, skipping checksums:

```bash
cd hermes/; bro -C -r ../SpyHunter-Operation_Hermes-ver1.pcap
```

The weird log is populated with oddities but none jumped out at me. From the Bro connections logs we see lots of web traffic, definitely surfing going on. As the file screams past me (forgot to page) I see some unencrypted chat and decide to look for the chat contents.

Slice and dice
---

* tcpdump

I used tcpdump to write smaller pcaps of the traffic for the suspect hosts from out steg tools and chat sessions.

```bash
## steg downloader, chat user
tcpdump -nnr SpyHunter-Operation_Hermes-ver1.pcap -w hermes-zero62.pcap 'host 192.168.0.62'
## steg download sites
tcpdump -nnr SpyHunter-Operation_Hermes-ver1.pcap -w hermes-24351.pcap 'host 69.160.243.51'
tcpdump -nnr SpyHunter-Operation_Hermes-ver1.pcap -w hermes-101016.pcap 'host 74.50.101.106'
## chat server
tcpdump -nnr SpyHunter-Operation_Hermes-ver1.pcap -w hermes-webchat.pcap 'host 144.76.244.204'
```

Focus on chat
---

* tcpdump, shell tools

Starting with tcpdump ASCII dump and then chaining shell tools to reformat the output I was able to pull out the chat room name, participants and chats which are quite helpful towards answering the scenario questions.

```bash
tcpdump -nnr hermes-webchat.pcap -v -A > webchat
egrep '"c","PRIVMSG"' webchat | sed -e 's/\[\[//' -e 's/\]\]//' | awk -F'[' '{print $2}' 
    | sed -e 's/\]//' > 10thRow-chatroom2
```

Python for webchat
---

I got much better results using Python on the web chat data in the packets. Thanks @scoob for help and his code is way better : [pastebin]

In [1]:
import re
#
chat_txt=[]

def dice_line (chatline):
    handle = re.sub(r'''\[\["c","PRIVMSG","''','',chatline.split('!')[0])
    msg = re.sub(r'\]\]\]\n','',chatline.split(",")[4])
    room = re.sub(r'\[','',chatline.split(",")[3])
    ip_addr = re.sub(r'''.*ip.(.*)"''',r'\1',chatline.split('!')[1].split(',')[0])
    return str(room + ": " + handle + " @ " + ip_addr + " " +  msg)


with open("webchat", "r") as f:
  for line in f.readlines():
    if re.search(r'"c","PRIVMSG",', line):
      chat_line = dice_line(line) ## slice and format here
      chat_txt.append(chat_line)

     
tenth_lines = chat_txt
## print "Got",len(tenth_lines),"lines"
for chat in tenth_lines:
  print(chat)

"#10thRow": LiquidTrace @ 62.210.84.4 "You figured out how to get our sample
"#10thRow": Saint99 @ 81.7.14.246 "Oh a smarty pants "
"#10thRow": LiquidTrace @ 62.210.84.4 "As we discussed we have more
"#10thRow": Saint99 @ 81.7.14.246 "One would have thought we just got lucky with your username here on freenode
"#10thRow": MonaSky @ 5.79.68.161 "Good.  Can not wait to do business with you"
"#10thRow": Saint99 @ 81.7.14.246 "I know the business you want to do"
"#10thRow": Ir0nH4ck @ 85.25.43.45 "LOL"
"#10thRow": Ir0nH4ck @ 85.25.43.45 "Watch out for Mona......"
"#10thRow": MonaSky @ 5.79.68.161 "What is that supposed to mean?"
"#10thRow": LiquidTrace @ 62.210.84.4 "YellowFin9999
"#10thRow": LiquidTrace @ 62.210.84.4 "You are late!!!"
"#10thRow": LiquidTrace @ 62.210.84.4 "Well at least we are making progress now."
"#10thRow": LiquidTrace @ 62.210.84.4 "First payment is due in 24 hours at the following account CH10 09260 00A209827341"
"#10thRow": LiquidTrace @ 62.210.84.4 "The second paym

Timelines
===

*...next up...*